# Texture and hyperchromatic areas feature exploration

In [1]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import flood_fill
from skimage import io, color, exposure, img_as_ubyte
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans, MeanShift

In [2]:
#Uncomment to install dependecy
#!pip install opencv-python 

In [3]:
#pip install scikit-image

### Declare variables and lists

In [3]:
# Images root directorie
data_dir = "cancerSeno_bw/train"

# Class list
classes = ["sano", "cancer"]

# Define angles and distanxes for GLCM (Gray-Level Co-Occurrence Matrix)
angles = [0]
distances = [1]

# Define normalization threshold
treshold = 120

# Define max number of clusters for segmentation
max_clusters = 5

# Initialize lists
# Store texture and density of pixels characteristics
glcm_features = []
densidades_pixeles_oscuros = []
num_cúmulos = []

# Store coordinates of origins in clusters
cluster_seeds = []

# Store loaded images
loaded_images = []

### Obtain images from directories

In [4]:
# Iterate over classes





health_dir = os.path.join(data_dir, classes[0])
ill_dir = os.path.join(data_dir, classes[1])
# Obtain list of image files from directory
health_file_list = os.listdir(health_dir)
ill_file_list = os.listdir(ill_dir)
    
# Store the first 3 images from current class
health_images_source = [os.path.join(health_dir, image_name) for image_name in health_file_list]
ill_images_source = [os.path.join(ill_dir, image_name) for image_name in ill_file_list]
    
# Add image paths to the loaded-images list
#loaded_images.extend(selected_images)

health_images = [io.imread(image_path) for image_path in health_images_source]




ill_images = [io.imread(image_path) for image_path in ill_images_source]






In [7]:
#Import required modules
from sklearn.decomposition import PCA
 
pca = PCA(n_components=3) # we need 2 principal components.

compressed_images = []

for i in health_images:
    
    converted_data = pca.fit_transform(i)
    compressed_images.append(converted_data)


ValueError: n_components=3 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [ ]:
compressed_images

In [14]:
ill_images

[array([[253, 252, 252, ..., 251, 251, 251],
        [253, 252, 252, ..., 251, 251, 251],
        [252, 252, 252, ..., 251, 251, 251],
        ...,
        [252, 252, 252, ...,  85,  72, 102],
        [252, 252, 251, ...,  87,  73,  69],
        [252, 252, 251, ..., 109, 106, 117]], dtype=uint8),
 array([[252, 252, 252, ..., 250, 196,  67],
        [252, 252, 252, ..., 248, 117, 128],
        [252, 252, 252, ..., 120, 129, 102],
        ...,
        [251, 251, 251, ..., 112, 116,  76],
        [251, 251, 251, ...,  66, 100, 161],
        [251, 251, 251, ..., 114, 242, 248]], dtype=uint8),
 array([[251, 251, 251, ..., 205, 249, 250],
        [251, 251, 250, ..., 233, 248, 249],
        [251, 251, 251, ..., 187, 249, 249],
        ...,
        [107,  99, 132, ..., 126, 210, 247],
        [ 72,  64, 125, ..., 107,  83, 123],
        [ 85,  91,  89, ..., 142, 124,  95]], dtype=uint8),
 array([[121, 122,  88, ...,  76, 102, 141],
        [127, 100, 146, ...,  99,  94,  96],
        [ 99, 12

In [26]:
health_x = []

for i in  health_images:
    for j in range(len(i)):
        ar = np.asarray(i[j])
        health_x.append(ar)

ill_x = []
for i in  ill_images:
    for j in range(len(i)):
        ar = np.asarray(i[j])
        ill_x.append(ar)

#health_x = np.array(health_x)
#ill_x = np.array(ill_x)

In [32]:
#health_x = np.array(health_x)
health_x

[array([187, 180, 194, 193, 184, 187, 175, 178, 195, 171, 145, 153, 138,
        149, 159, 167, 171, 162, 153, 147, 151, 155, 156, 154, 144, 154,
        173, 131, 188, 218, 231, 230, 240, 246, 248, 245, 247, 246, 233,
        209, 202, 196, 178, 198, 211, 223, 221, 229, 202, 224], dtype=uint8),
 array([170, 163, 177, 162, 158, 146, 153, 147, 149, 156, 160, 133, 152,
        145, 153, 151, 155, 150, 158, 153, 146, 141, 143, 161, 138, 152,
        146, 153, 166, 164, 215, 167, 214, 236, 247, 248, 247, 249, 239,
        221, 208, 193, 195, 196, 211, 174, 181, 191, 199, 221], dtype=uint8),
 array([200, 183, 159, 160, 160, 157, 152, 152, 151, 144, 135, 122, 137,
        153, 164, 167, 168, 178, 185, 162, 147, 140, 169, 160, 177, 178,
        167, 171, 203, 149, 157, 224, 213, 236, 244, 239, 202, 175, 167,
        157, 164, 177, 206, 230, 223, 225, 236, 199, 169, 188], dtype=uint8),
 array([184, 186, 188, 181, 176, 189, 191, 162, 154, 154, 167, 174, 183,
        200, 198, 184, 168, 179, 183

In [31]:
{len(e) for e in health_x}

{1, 2, 5, 11, 12, 16, 21, 22, 26, 31, 32, 36, 41, 42, 46, 50}

In [21]:

import pickle
with open('gray_large_health_list.pickle','wb') as file:
    pickle.dump(health_x, file)

In [28]:
ill_x

[array([253, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
        252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 251, 251, 251,
        252, 252, 252, 252, 252, 252, 251, 251, 251, 252, 251, 251, 250,
        250, 249, 249, 249, 250, 251, 251, 251, 251, 251, 251], dtype=uint8),
 array([253, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
        252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 251, 251, 251,
        252, 252, 252, 252, 252, 252, 251, 251, 251, 251, 252, 251, 251,
        250, 250, 249, 250, 252, 251, 251, 251, 251, 251, 251], dtype=uint8),
 array([252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
        252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 251, 251, 251,
        252, 253, 252, 252, 252, 252, 252, 251, 251, 251, 251, 251, 251,
        251, 251, 251, 251, 251, 251, 251, 251, 251, 251, 251], dtype=uint8),
 array([252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
        252, 252, 252, 252, 252, 252

In [22]:
with open('gray_large_ill_list.pickle','wb') as file:
    pickle.dump(ill_x, file)